In [22]:
import sys, os
sys.path.append(os.path.realpath("/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages"))
sys.path.append(os.path.realpath("../"))

import util_funcs
from importlib import reload
reload(util_funcs)
from copy import deepcopy as cp

import data_reader as read
import pandas as pd


reload(read)

edfRawData = read.EdfDataset("train", "01_tcp_ar", num_files=1, n_process=7)
edfFFTData = read.EdfFFTDatasetTransformer(edfRawData, window_size=pd.Timedelta(seconds=1), precache=False, n_process=7)

In [2]:
import multiprocessing as mp
mp.cpu_count()

28

In [3]:
data = edfRawData[0:21]

Starting 7 processes
retrieving: 0


In [ ]:
fftData = edfFFTData[0]

Starting 7 processes
retrieving: 0


In [21]:
fftData[0][1].shape

IndexError: list index out of range

In [15]:
333847/1305

255.8214559386973

In [5]:
data[0][0].index.max() / pd.Timedelta(seconds=1/256)

334101.0

In [23]:
original_data = data[0]

In [10]:
import numpy as np

In [21]:
original_data

(                 EEG FP1-REF  EEG FP2-REF  EEG F3-REF  EEG F4-REF  EEG C3-REF  \
 00:00:00          -44.552129   -87.124117   28.232238   -4.116370   34.335749   
 00:00:00.003906   -46.688358   -85.750827   27.316711   -5.642248   34.030573   
 00:00:00.007812   -50.808228   -94.143155   26.706360  -10.525056   32.809871   
 00:00:00.011718   -50.045289   -97.042323   25.333070  -16.781155   32.962459   
 00:00:00.015624   -51.876342   -96.584559   24.722719  -15.102689   33.420222   
 00:00:00.019530   -55.385861  -101.925131   18.771796  -16.628567   28.690001   
 00:00:00.023436   -55.996212   -91.396575   26.096009  -16.628567   30.826230   
 00:00:00.027342   -54.622922   -97.957849   15.872628  -11.593171   34.335749   
 00:00:00.031248   -53.859983  -111.538161   25.485658  -32.955459   36.624565   
 00:00:00.035154   -63.167837  -120.235664   19.992498  -19.222559   35.861626   
 00:00:00.039060   -63.625601  -121.151190   21.976139  -23.342429   41.049610   
 00:00:00.042966

In [31]:
window_count_size = int(pd.Timedelta(seconds = 1) / pd.Timedelta(seconds=util_funcs.COMMON_DELTA))

fft_data = np.nan_to_num(np.abs(np.fft.fft(original_data[0].values, axis=0)))
fft_data_windows = util_funcs.np_rolling_window(np.array(fft_data.T), window_count_size)
fft_data_windows[:,list(range(0, fft_data_windows.shape[1], window_count_size))]
# fft_data = np.abs(
#                 np.fft.fft(
#                     util_funcs.np_rolling_window(np.array(fft_data.T), window_count_size),
#                     axis=2)) #channel, window num, frequencies

(30, 1305, 256)

In [7]:
util_funcs.np_rolling_window(np.array(fft_data.T), window_count_size).shape

(30, 333847, 256)

In [10]:
data[0].index

TimedeltaIndex([       '00:00:00', '00:00:00.003906', '00:00:00.007812',
                '00:00:00.011718', '00:00:00.015624', '00:00:00.019530',
                '00:00:00.023436', '00:00:00.027342', '00:00:00.031248',
                '00:00:00.035154',
                ...
                '00:21:44.963352', '00:21:44.967258', '00:21:44.971164',
                '00:21:44.975070', '00:21:44.978976', '00:21:44.982882',
                '00:21:44.986788', '00:21:44.990694', '00:21:44.994600',
                '00:21:44.998506'],
               dtype='timedelta64[ns]', length=334102, freq='3906U')

In [4]:
data[0]

(     EEG FP1-REF   EEG FP2-REF    EEG F3-REF    EEG F4-REF    EEG C3-REF  \
 0   1.722230e+09  7.612760e+08  2.589363e+08  1.887984e+08  1.343045e+08   
 1   4.533975e+08  9.602734e+07  3.592292e+07  3.066479e+07  2.008086e+07   
 2   1.512986e+08  4.849421e+07  2.063142e+07  1.985958e+07  1.603775e+07   
 3   1.875688e+08  3.106360e+07  1.684585e+07  1.570359e+07  1.304105e+07   
 4   1.274519e+08  2.316599e+07  1.398178e+07  1.369064e+07  1.144430e+07   
 5   1.604666e+08  1.749697e+07  1.213084e+07  1.195733e+07  1.003461e+07   
 6   1.440705e+08  1.533564e+07  1.007001e+07  1.067791e+07  9.381422e+06   
 7   1.580244e+08  1.401605e+07  1.074924e+07  1.023057e+07  9.872164e+06   
 8   1.121496e+08  1.665934e+07  1.377455e+07  1.246416e+07  1.676608e+07   
 9   1.511992e+08  2.476634e+07  1.645324e+07  1.912685e+07  1.935413e+07   
 10  1.074652e+08  1.522933e+07  1.101756e+07  1.210168e+07  1.012398e+07   
 11  9.235826e+07  1.161382e+07  8.513492e+06  9.453683e+06  7.960644e+06   

In [39]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
# from seq2seq.models import Seq2Seq

# model = Seq2Seq(batch_input_shape=(16, 7, 5), hidden_dim=10, output_length=8, output_dim=20, depth=4)
# model.compile(loss='mse', optimizer='rmsprop')

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ms994/miniconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ms994/miniconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/ms994/miniconda3/envs/keras/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/ms994/miniconda3/envs/keras/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: /lib64/libm.so.6: version `GLIBC_2.23' not found (required by 